In [1]:
#Classification
from mlbox.preprocessing import Reader
from mlbox.preprocessing import Drift_thresholder
from mlbox.optimisation import Optimiser
from mlbox.prediction import Predictor
import csv 


# Paths to the train set and the test set.
paths = ["../../Data/titanic/processed/train.csv","../../Data/titanic/processed/x_test.csv"]
# Name of the feature to predict.
# This columns should only be present in the train set.
target_name = "Survived"

# Reading and cleaning all files
# Declare a reader for csv files
rd = Reader(sep=',')
# Return a dictionnary containing three entries
# dict["train"] contains training samples withtout target columns
# dict["test"] contains testing elements withtout target columns
# dict["target"] contains target columns for training samples.
data = rd.train_test_split(paths, target_name)

dft = Drift_thresholder()
data = dft.fit_transform(data)

# Tuning
# Declare an optimiser. Scoring possibilities for classification lie in :
# {"accuracy", "roc_auc", "f1", "neg_log_loss", "precision", "recall"}
opt = Optimiser(scoring='accuracy', n_folds=3)
opt.evaluate(None, data)

# Space of hyperparameters
# The keys must respect the following syntax : "enc__param".
#   "enc" = "ne" for na encoder
#   "enc" = "ce" for categorical encoder
#   "enc" = "fs" for feature selector [OPTIONAL]
#   "enc" = "stck"+str(i) to add layer n°i of meta-features [OPTIONAL]
#   "enc" = "est" for the final estimator
#   "param" : a correct associated parameter for each step.
#   Ex: "max_depth" for "enc"="est", ...
# The values must respect the syntax: {"search":strategy,"space":list}
#   "strategy" = "choice" or "uniform". Default = "choice"
#   list : a list of values to be tested if strategy="choice".
#   Else, list = [value_min, value_max].
# Available strategies for ne_numerical_strategy are either an integer, a float
#   or in {'mean', 'median', "most_frequent"}
# Available strategies for ce_strategy are:
#   {"label_encoding", "dummification", "random_projection", entity_embedding"}
space = {'ne__numerical_strategy': {"search": "choice", "space": [0]},
         'ce__strategy': {"search": "choice",
                          "space": ["label_encoding",
                                    "random_projection",
                                    "entity_embedding"]},
         'fs__threshold': {"search": "uniform",
                           "space": [0.01, 0.3]},
         'est__max_depth': {"search": "choice",
                            "space": [3, 4, 5, 6, 7]}

         }


#



reading csv : train.csv ...
cleaning data ...
CPU time: 0.14305758476257324 seconds

reading csv : x_test.csv ...
cleaning data ...
CPU time: 0.05038189888000488 seconds

> Number of common features : 17

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 0
> Number of numerical features: 17
> Number of training samples : 596
> Number of test samples : 295

> You have no missing values on train set...

> Task : classification
0.0    355
1.0    241
Name: Survived, dtype: int64

encoding target ...

computing drifts ...
CPU time: 0.4463634490966797 seconds

> Top 10 drifts

('Age', 0.08351279474242057)
('SibSp', 0.05880442146022191)
('Fare', 0.057527600192988526)
('Sex_male', 0.05660923901575865)
('Title_Mr', 0.04619353636611545)
('PassengerId', 0.045880809243704856)
('Title_Master', 0.03666179876630493)
('Embarked_C', 0.0271

/opt/conda/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")



MEAN SCORE : accuracy = 0.7852139485305315
VARIANCE : 0.020466045505150618 (fold 1 = 0.8140703517587939, fold 2 = 0.7688442211055276, fold 3 = 0.7727272727272727)
CPU time: 1.145263910293579 seconds



In [2]:
# Optimises hyper-parameters of the whole Pipeline with a given scoring
# function. Algorithm used to optimize : Tree Parzen Estimator.
best = opt.optimise(space,data,40)
print("Final results : " ,opt.evaluate(best, data))

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'entity_embedding'}      
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.2634696989628652}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 4, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



MEAN SCORE : accuracy = 0.7868551511767592            
VARIANCE : 0.047673159561935845 (fold 1 = 0.8542713567839196, fold 2 = 0.7537688442211056, fold 3 = 0.7525252525252525)
CPU time: 1.8632047176361084 seconds                  
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}      
>>> CA ENCODER :{'strategy': 'label_encoding'}                                   
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.23419186577138557}       
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 3, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'si

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



MEAN SCORE : accuracy = 0.8086476151802785
VARIANCE : 0.03960086924490679 (fold 1 = 0.8592964824120602, fold 2 = 0.8040201005025126, fold 3 = 0.7626262626262627)
CPU time: 0.8350939750671387 seconds                                             
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}      
>>> CA ENCODER :{'strategy': 'label_encoding'}                                   
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.2649965268974138}        
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 5, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lam

...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



MEAN SCORE : accuracy = 0.8086476151802785                                        
VARIANCE : 0.03960086924490679 (fold 1 = 0.8592964824120602, fold 2 = 0.8040201005025126, fold 3 = 0.7626262626262627)
CPU time: 0.7881207466125488 seconds                                              
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}       
>>> CA ENCODER :{'strategy': 'label_encoding'}                                    
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.07478772329720093}        
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 4, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'ran

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



MEAN SCORE : accuracy = 0.7868551511767592                                        
VARIANCE : 0.047673159561935845 (fold 1 = 0.8542713567839196, fold 2 = 0.7537688442211056, fold 3 = 0.7525252525252525)
CPU time: 0.9580309391021729 seconds                                              
100%|██████████| 40/40 [00:49<00:00,  1.23s/trial, best loss: -0.8086476151802785]


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ BEST HYPER-PARAMETERS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

{'ce__strategy': 'label_encoding', 'est__max_depth': 3, 'fs__threshold': 0.23419186577138557, 'ne__numerical_strategy': 0}

##################################################### testing hyper-parameters... ########

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [3]:
from mlbox.prediction import *
pred=Predictor()
pred.fit_predict(best,data)


fitting the pipeline ...
CPU time: 0.26691675186157227 seconds

> Feature importances dumped into directory : save

predicting ...
CPU time: 0.016614913940429688 seconds

> Overview on predictions : 

        0.0       1.0  Survived_predicted
0  0.623678  0.376322                   0
1  0.014289  0.985711                   1
2  0.977290  0.022710                   0
3  0.619419  0.380581                   0
4  0.163809  0.836191                   1
5  0.979954  0.020046                   0
6  0.191114  0.808886                   1
7  0.934425  0.065575                   0
8  0.886010  0.113990                   0
9  0.904934  0.095066                   0

dumping predictions into directory : save ...


In [5]:
predictions = pd.read_csv("save/Survived_predictions.csv")
predictions = predictions.Survived_predicted
y_test = pd.read_csv("../../Data/titanic/processed/y_test.csv")

In [6]:
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
fpr, tpr, _ = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
plt.savefig("ML_Box_Titanic_ROC.pdf")